# Agenda
1. Grouping and pivot tables
2. More with multi-indexes (e.g., stack and unstack)
3. Joining, merging, and concatenating
4. Working with text



In [3]:
import pandas as pd

filename = 'taxi.csv'

df = pd.read_csv(filename,
                usecols=['VendorID', 'passenger_count', 'trip_distance',
                         'total_amount', 'payment_type'])

In [4]:
df.head()

,VendorID,passenger_count,trip_distance,payment_type,total_amount
0,2,1,1.63,2,17.80
1,2,1,0.46,1,8.30
2,2,1,0.87,1,11.00
3,2,1,2.13,1,17.16
4,1,1,1.40,2,10.30


In [5]:
# I want to know how much people paid, on average (mean) for their taxi rides

df['total_amount'].mean()

np.float64(17.552472247224728)

In [7]:
# I want to know how much people paid, on average, for their taxi rides where there were 0 passengers

(
    df.loc[
        df['passenger_count'] == 0,
        'total_amount'
       ]
    .mean()
)

np.float64(25.57)

In [8]:
# I want to know how much people paid, on average, for their taxi rides where there were 1 passenger

(
    df.loc[
        df['passenger_count'] == 1,
        'total_amount'
       ]
    .mean()
)

np.float64(17.368569446371584)

In [9]:
# I want to know how much people paid, on average, for their taxi rides where there were 2 passengers

(
    df.loc[
        df['passenger_count'] == 2,
        'total_amount'
       ]
    .mean()
)

np.float64(18.406306169078444)